## Project 2 (Intermediate) — Multi‑User Org‑Scoped App (Tasks / Mini‑CRM)  
### Workbook Guidelines (What to build, why, how to verify) — not spoonfeeding full code

This project is where learners move from “single-user CRUD” to **real multi-user,
multi-tenant patterns**:

- tenant/org scoping (no cross-org data leaks)
- roles and object-level permissions
- audit fields and event history
- filter/search/pagination with preserved state
- exports and internal tooling patterns
- background jobs + (optional) realtime notifications

Treat this like a miniature SaaS module.

---

# 1) Project Goal (What you’re building)

Build an **Organization‑scoped Tasks app** where:

- Users belong to organizations (tenants) via memberships.
- Tasks belong to one organization.
- Users can create, view, and update tasks **only** within orgs they belong to.
- Role rules and object-level rules control editing, deleting, and exporting.
- The system maintains audit fields and (optionally) event logs.
- Exports are safe, scoped, and production-minded (sync baseline; async stretch).

---

# 2) Target Skills (What this project teaches)

You should practice:

- Multi-tenancy discipline:
  - tenant identification from URL
  - queryset scoping always includes `organization`
  - tests that catch tenant leaks (ID guessing)
- Authorization at scale:
  - RBAC (ADMIN vs MEMBER)
  - ABAC/object rules (creator/assignee can edit)
  - centralized policy functions
- Maintainable architecture:
  - selectors/services/policies
  - minimal fat views
- UX patterns:
  - filter form via GET
  - pagination that preserves filters
  - PRG for create/edit/delete flows
- Operational patterns:
  - CSV export (with permission checks)
  - background export job model (stretch)
  - audit logging / TaskEvent history (stretch)

---

# 3) Requirements (User Stories + Acceptance Criteria)

## 3.1 Organization + Membership
1. **Organization exists**
   - Each org has `name` + `slug` (slug used in URLs).
2. **Membership exists**
   - A user can be a member of multiple orgs.
   - Membership has a `role` (at minimum ADMIN, MEMBER).
3. **Unique membership**
   - (org, user) must be unique at DB level.

Acceptance criteria:
- attempting to create a second membership row for same (org, user) fails.

---

## 3.2 Task core
4. **Task belongs to org**
   - Every task has `organization` FK, cannot be null.
5. **CRUD**
   - Org member can list tasks in org.
   - Org member can view a task in org.
   - Org member can create a task in org.
   - Update and delete rules defined below.
6. **Task fields**
   - title (required), description (optional)
   - status (OPEN/IN_PROGRESS/DONE/CANCELED)
   - priority (LOW/MEDIUM/HIGH/URGENT or numeric choices)
   - due_date optional
   - assigned_to optional (FK to user)

Acceptance criteria:
- all pages work without leaking tasks across orgs
- filters and pagination work with realistic dataset sizes

---

## 3.3 Audit fields
7. **Audit fields exist**
   - created_by (required)
   - updated_by (required)
   - created_at/updated_at timestamps

Acceptance criteria:
- created_by is set to the actor on create (server-side)
- updated_by updates on every edit (server-side)
- users cannot set these fields via client input

---

## 3.4 Authorization (must be explicit)
Define the project’s authorization rules as a matrix.

Recommended baseline rules:

### Roles
- ADMIN:
  - can view/create/edit/delete tasks in org
  - can export tasks
- MEMBER:
  - can view/create tasks in org
  - can edit tasks only if:
    - they created it OR
    - it is assigned to them
  - cannot delete tasks
  - cannot export tasks (optional; you may allow export for MANAGER role as stretch)

Acceptance criteria:
- unauthorized edits return 403
- cross-org access returns 404 (recommended policy for tenant boundary)

---

## 3.5 Filtering, searching, and pagination
8. **Task list filters (GET query params)**
   - `q`: search title/description
   - `status`
   - `priority`
   - `assigned_to`:
     - `me` means assigned_to = current user
     - or username match (optional)

9. **Pagination**
   - list is paginated (never unbounded)
   - filter params persist across pages
   - invalid `page` does not crash

Acceptance criteria:
- `/orgs/acme/tasks/?status=open&page=2` works
- changing filter does not break pagination
- querystring preservation is correct (no duplicated page params)

---

## 3.6 CSV export
10. **Export endpoint exists**
   - exports tasks to CSV with the same filters as the list view
   - scoped by org
   - permission-protected (ADMIN only baseline)

Acceptance criteria:
- member gets 403 (or 404 by policy)
- admin gets a CSV download with Content-Disposition header
- exported rows match filtered list semantics

---

## 3.7 Optional history (stretch but recommended)
11. **TaskEvent or AuditLog**
   - record important events:
     - created
     - status_changed
     - assignee_changed

Acceptance criteria:
- status change creates an event row
- event log is scoped to task/org
- events can be shown on task detail (optional)

---

# 4) Data Model Guidelines (Minimum viable schema)

## 4.1 Organization
Fields:
- name
- slug unique
- created_at

## 4.2 Membership (join model)
Fields:
- organization FK
- user FK
- role (choices)

Constraints:
- unique(org, user)

Indexes:
- (organization, role)
- (user)

## 4.3 Task
Fields:
- organization FK
- title, description
- status, priority, due_date
- assigned_to FK nullable
- created_by FK, updated_by FK
- timestamps

Indexes (guidelines):
- (organization, status, created_at desc) for list
- (organization, assigned_to) for “my tasks”
- (organization, due_date) for due sorting/filters

## 4.4 TaskExportJob (stretch, for async export)
Fields:
- organization FK
- created_by FK
- filters JSON
- status (pending/running/done/failed)
- file
- timestamps
- error text

---

# 5) URL Design Guidelines (Org‑Scoped Routing)

Recommended URL scheme:

- `/orgs/<org_slug>/tasks/` list
- `/orgs/<org_slug>/tasks/new/` create
- `/orgs/<org_slug>/tasks/<id>/` detail
- `/orgs/<org_slug>/tasks/<id>/edit/` edit
- `/orgs/<org_slug>/tasks/<id>/delete/` delete (admin only)
- `/orgs/<org_slug>/tasks/export.csv` export (admin only)

**Rules**
- Org slug must be present in URL for all tenant-scoped routes.
- Never accept org id from form input for scoping.
- Use URL namespacing: `tasks:list`, `tasks:detail`, etc.

---

# 6) Implementation Plan (Milestones)

## Milestone 1 — Org + Membership + Admin manageability
Deliverables:
- models + migrations
- admin screens for Organization + Membership
- ability to create org and add members

Definition of Done:
- you can create org “acme”
- you can add two users with roles admin/member
- uniqueness constraint prevents duplicate membership

---

## Milestone 2 — Task CRUD with strict scoping
Deliverables:
- task list/detail/create/edit/delete
- every query includes org scope

Definition of Done:
- a user who is not a member gets 404 for any org route
- a member can list tasks in their org
- guessing another org’s task id doesn’t work

Guidelines:
- use a helper/service like `get_org_for_user_or_404(user, org_slug)`
- in every task retrieval, do:
  - `get_object_or_404(Task, id=task_id, organization=org)`

---

## Milestone 3 — Authorization rules
Deliverables:
- role checks implemented consistently
- object-level edit rules implemented

Definition of Done:
- member cannot edit tasks they do not own/aren’t assigned
- admin can edit any task
- member cannot delete
- admin can delete

Guidelines:
- centralize rules in policy functions (recommended):
  - `tasks/policies.py` or `tasks/permissions.py`

---

## Milestone 4 — Filtering + pagination UX
Deliverables:
- filter form with GET
- pagination preserving query params
- stable ordering

Definition of Done:
- filters combine (q + status + priority + assigned_to)
- pagination links preserve filter state
- invalid page does not crash

Guidelines:
- use a Form for filter validation (clean parsing for ints and choices)
- use a shared querystring builder tag to avoid duplicated page params

---

## Milestone 5 — CSV export
Deliverables:
- export endpoint
- permission rules
- same filters as list view

Definition of Done:
- download works
- content matches UI filters
- permission enforced and tested

Guidelines:
- never export without org filter
- stream or chunk if dataset can be large (iterator)

---

## Milestone 6 (Stretch) — Background export jobs + notifications
Deliverables:
- TaskExportJob model
- start/status/download endpoints OR admin tool
- Celery task generates CSV asynchronously
- optional WebSocket event broadcast

Definition of Done:
- export start returns quickly (202 or redirect)
- status endpoint shows progress/state
- download works only after job is DONE
- job is scoped to org and creator/admin permissions

---

# 7) Testing Guidelines (Minimum set that prevents expensive bugs)

This project must have tests for:

## 7.1 Tenant isolation regression tests (highest value)
- Non-member cannot access org list (404)
- Non-member cannot access task detail even if they guess ID (404)
- Member of org A cannot access a task in org B even using org A URL (404)

## 7.2 Authorization tests
- Member can edit assigned task
- Member cannot edit not-assigned/not-created task (403)
- Member cannot export (403)
- Admin can export (200)

## 7.3 Filtering/pagination tests
- filtering returns expected results
- invalid page handled gracefully
- query params preserved in pagination links (smoke check)

## 7.4 CSV export tests
- Content-Type is `text/csv`
- Content-Disposition includes attachment filename
- output includes expected header row

## 7.5 Audit field tests
- created_by/updated_by set correctly
- client cannot override created_by/updated_by via POST payload (server must ignore)

---

# 8) Security Guidelines (Project 2 is a security project)

Must-do rules:

- **Never** trust org identifiers from client body.
- Derive org from URL and validate membership server-side.
- All state-changing actions must be POST with CSRF token (HTML).
- API writes must require authentication and permission checks.
- Exports are sensitive: protect access and log/audit export actions.

Optional but recommended:
- rate limit export endpoint
- audit log “export started/downloaded”

---

# 9) Performance Guidelines (Realistic)

- List view must paginate.
- Use `select_related` for FK fields displayed:
  - assigned_to
  - created_by
  - updated_by
- Use indexes aligned with list filters:
  - (organization, status, created_at)
- CSV export should use `.iterator(chunk_size=...)` to avoid memory blow-up.

---

# 10) “Definition of Done” (Project 2 Completion Gate)

Project 2 is complete when:

- [ ] Orgs + memberships exist with unique constraint
- [ ] Tasks are always scoped by org in queries
- [ ] Member vs admin permissions enforced consistently
- [ ] Filters + pagination work and preserve state
- [ ] CSV export exists and is permission-protected
- [ ] Tests cover tenant leaks + permission rules + export
- [ ] Lint/format/test pipeline passes

---

# 11) Stretch Goals (Pick 2–5)

1. Add role `MANAGER` with:
   - can export
   - cannot delete
2. Add TaskEvent history + show on task detail
3. Add “My Tasks” quick filter (assigned_to=me)
4. Add background export jobs (Celery) with TaskExportJob + download link
5. Add WebSocket notifications for task changes (Channels)
6. Add audit logging for sensitive actions (export, membership role changes)
7. Add “members management” UI (non-admin) with invite flow (email) (advanced)

---

# 12) Common Failure Modes (and what to check)

- “User can see other org’s tasks by guessing ID”
  - Your `get_object_or_404` is missing `organization=org`.

- “Export returns tasks from multiple orgs”
  - Export query not scoped by org or filters duplicated incorrectly.

- “Member can delete tasks”
  - Delete view not enforcing policy.

- “Pagination loses filters or duplicates page param”
  - Querystring builder missing or incorrect; rebuild querystring excluding page then set page.

- “assigned_to filter breaks”
  - Treat `assigned_to=me` and `assigned_to=<username>` differently; validate safely.

- “CSV export is slow / memory heavy”
  - Use `.iterator()` and don’t build a huge list in memory.

---